In [1]:
import numpy as np
from cobra.io import read_sbml_model

In [2]:
model = read_sbml_model('iML1515.xml.gz')

We want to look at the model. See if it has fluxes

In [3]:
solution = model.optimize()
solution.fluxes

CYTDK2     0.00000
XPPT       0.00000
HXPRT      0.00000
NDPK5      0.00000
SHK3Dr     0.33424
            ...   
MPTS       0.00000
MOCOS      0.00000
BMOGDS2    0.00000
FESD2s     0.00000
OCTNLL     0.00000
Name: fluxes, Length: 2712, dtype: float64

Checking what its objective is

In [4]:
model.summary()

We want to find the metabolite Purtescine in the model

In [5]:
for metabolite in model.metabolites.query('Putrescine', 'name'):
    print(metabolite.name)
    print(metabolite.id)

Putrescine
ptrc_p
Putrescine
ptrc_c
Putrescine
ptrc_e


In [6]:
model.metabolites.get_by_id('ptrc_c') # Putrescine

Metabolite identifier,ptrc_c
Name,Putrescine
Memory address,0x07fc802ce6d68
Formula,C4H14N2
Compartment,c
In 9 reaction(s),"PTRCTA, AGMT, BIOMASS_Ec_iML1515_WT_75p37M, PTRCabcpp, PTRCt2pp, GGPTRCS, PTRCORNt7pp, ORNDC, SPMS"


The reaction that produces purtescine from onithine is native to E.coli

In [7]:
model.reactions.ORNDC

Reaction identifier,ORNDC
Name,Ornithine Decarboxylase
Memory address,0x07fc80190c780
Stoichiometry,h_c + orn_c --> co2_c + ptrc_c H+ + Ornithine --> CO2 CO2 + Putrescine
GPR,b2965 or b0693
Lower bound,0.0
Upper bound,1000.0


We want to check the flux

In [8]:
with model:
    model.objective = model.reactions.ORNDC
    print(model.optimize().objective_value)  # We produce Purtescine

41.09128205128201


In [9]:
from cobra import Reaction, Metabolite

To make E. Coli produce putrescine we need to add the reaction of the enzyme 2.6.1.13.
This is the same as we added in C. Glutamicum.

In [10]:
new_reaction_1 = Reaction('OAT')  # OAT -> (o)rnithine (a)mino(t)ransferase

In [11]:
for metabolite in model.metabolites.query('5-semialdehyde', 'name'):
    print(metabolite.name)
    print(metabolite.id)

N-Acetyl-L-glutamate 5-semialdehyde
acg5sa_c
N2-Succinyl-L-glutamate 5-semialdehyde
sucgsa_c
L-Glutamate 5-semialdehyde
glu5sa_c


In [12]:
model.metabolites.get_by_id('glu5sa_c') # L-glutamate 5-semialdehyde

Metabolite identifier,glu5sa_c
Name,L-Glutamate 5-semialdehyde
Memory address,0x07fc802e81e48
Formula,C5H9NO3
Compartment,c
In 2 reaction(s),"G5SD, G5SADs"


We see that the two reactions L-glutamate 5-semialdehyde is part of correspond to the ones seen in the KEGG pathway map. We now need to find the other metabolites that play a role in conversion of L-glutamate 5-semialdehyde to ornithine.

In [13]:
for metabolite in model.metabolites.query('Ornithine', 'name'):
    print(metabolite.name)
    print(metabolite.id)

Ornithine
orn_e
Ornithine
orn_p
Ornithine
orn_c


In [14]:
model.metabolites.get_by_id('orn_c') # Ornithine

Metabolite identifier,orn_c
Name,Ornithine
Memory address,0x07fc802d9b7f0
Formula,C5H13N2O2
Compartment,c
In 6 reaction(s),"PTRCORNt7pp, ARGORNt7pp, ORNDC, OCBT, ACODA, ORNabcpp"


In [15]:
for metabolite in model.metabolites.query('Oxoglutarate', 'name'):
    print(metabolite.name)
    print(metabolite.id)

2-Oxoglutarate
akg_e
2-Oxoglutarate
akg_c
2-Oxoglutarate
akg_p


In [16]:
model.metabolites.get_by_id('akg_c') # 2-Oxoglutarate

Metabolite identifier,akg_c
Name,2-Oxoglutarate
Memory address,0x07fc8030e04e0
Formula,C5H4O5
Compartment,c
In 28 reaction(s),"VALTA, CYSTA, OHPBAT, AHGDx, ABTA, ILETA, TDPAGTA, ACOTA, AKGDH, PHETA1, UDPKAAT, SDPTA, SHGO, SOTA, PTRCTA, AKGt2rpp, ALATA_L, ARHGDx, PSERT, LEUTAi, GLUDy, ASPTA, TAUDO, SEPHCHCS, HSTPT, GLUSy, T..."


In [17]:
for metabolite in model.metabolites.query('L-Glutamate', 'name'):
    print(metabolite.name)
    print(metabolite.id)

L-Glutamate 1-semialdehyde
glu1sa_c
L-Glutamate
glu__L_c
L-Glutamate 5-semialdehyde
glu5sa_c
L-Glutamate
glu__L_e
L-Glutamate
glu__L_p
L-Glutamate 5-phosphate
glu5p_c


In [18]:
model.metabolites.get_by_id('glu__L_c') # L-Glutamate

Metabolite identifier,glu__L_c
Name,L-Glutamate
Memory address,0x07fc802e7ddd8
Formula,C5H8NO4
Compartment,c
In 51 reaction(s),"GGGABAH, GLUABUTt7pp, IG3PS, GLUPRT, GGPTRCS, GF6PTA, ILETA, LALGP, CBPS, PHETA1, GLUt4pp, ACGS, PTRCTA, GLUabcpp, ADCS, PSERT, GLUTRS, GLNS, GLUN, PRFGS, SGDS, TYRTA, GLUR, GLU5K, VALTA, GMPS2, CY..."


Now we add the reaction to produce ornithine

In [19]:
new_reaction_1.add_metabolites({model.metabolites.get_by_id('glu5sa_c'): -1, # L-glutamate 5-semialdehyde
                              model.metabolites.get_by_id('glu__L_c'): -1, # L-glutamate 
                              model.metabolites.get_by_id('orn_c'): 1, # L-ornithine
                              model.metabolites.get_by_id('akg_c'): 1 # 2-oxoglutarate
                             })

In [20]:
print(new_reaction_1.build_reaction_string())

glu5sa_c + glu__L_c --> akg_c + orn_c


In [21]:
model.add_reactions([new_reaction_1])
model.reactions.OAT

Reaction identifier,OAT
Name,
Memory address,0x07fc8027d0748
Stoichiometry,glu5sa_c + glu__L_c --> akg_c + orn_c L-Glutamate 5-semialdehyde + L-Glutamate --> 2-Oxoglutarate + Ornithine
GPR,
Lower bound,0.0
Upper bound,1000.0


In [22]:
with model:
    model.objective = model.reactions.OAT
    print(model.optimize().objective_value)  # We produce L-ornithine

53.0233333333333


We want to see production rate of putrescine now after we have added a new path via glutamate

In [23]:
with model:
    model.objective = model.reactions.ORNDC
    print(model.optimize().objective_value)  # We produce Purtescine

45.78742857142852


We see that the production rate of putrescine increases from 41.09 to 45.79

# Finding theoretical yield

We want to add a reaction that excretes putrescine

In [33]:
model.add_boundary(model.metabolites.ptrc_c, type='demand')

Reaction identifier,DM_ptrc_c
Name,Putrescine demand
Memory address,0x07fc801b35f98
Stoichiometry,ptrc_c --> Putrescine -->
GPR,
Lower bound,0
Upper bound,1000.0


We want to look at the medium composition

In [29]:
model.medium

{'EX_pi_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_slnt_e': 1000.0}

We find the maximal theoretical yield of putrescine both before and after adding our reaction OAT. 

In [36]:
medium = model.medium
with model:
    model.reactions.OAT.bounds = 0,0
    model.medium = medium
    model.objective = model.reactions.DM_ptrc_c
    put_production = model.optimize().objective_value
    print("Max. putriscine production [mmol gDW^-1 h^-1]:", put_production)
    print("Theoretical max. yield [mmol-put / mmol-glc]:", put_production / (-1*model.reactions.EX_glc__D_e.flux))

Max. putriscine production [mmol gDW^-1 h^-1]: 9.65333333333333
Theoretical max. yield [mmol-put / mmol-glc]: 0.965333333333333


In [37]:
medium = model.medium
with model:
    model.medium = medium
    model.objective = model.reactions.DM_ptrc_c
    put_production = model.optimize().objective_value
    print("Max. putriscine production [mmol gDW^-1 h^-1]:", put_production)
    print("Theoretical max. yield [mmol-put / mmol-glc]:", put_production / (-1*model.reactions.EX_glc__D_e.flux))

Max. putriscine production [mmol gDW^-1 h^-1]: 10.019104477611933
Theoretical max. yield [mmol-put / mmol-glc]: 1.0019104477611933


We see an increase in theoretical yield.